# 🛠️ Tool Calling con LangChain e LLM

## 🎯 Obiettivo

Consentire a un **LLM di rispondere a domande** a cui non può rispondere autonomamente, **delegando ad API esterne** tramite strumenti strutturati.

---

## 🤔 Perché usare il Tool Calling?

| Scenario                      | Tecnica        | Quando usarla                                 |
| ----------------------------- | -------------- | --------------------------------------------- |
| Domanda fuori contesto LLM    | ❌ No risposta  | Limite del knowledge cutoff                   |
| Recupero da documenti statici | ✅ RAG          | Info interne, storiche, aziendali             |
| Accesso a dati in tempo reale | ✅ Tool Calling | Meteo, API esterne, disponibilità, web search |

---

## 🧪 Esempio: Meteo e disponibilità all’aperto

### 🧭 Obiettivo

Rispondere a:

> “Che tempo farà oggi a Monaco? Avete ancora posti disponibili all’aperto?”

---

## ⚙️ Step 1 – Istanziazione del modello

```python
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
```

In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [1]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI()

llm.invoke("How will the weather be in munich today?")

AIMessage(content='I am unable to provide real-time weather updates. I recommend checking a reliable weather website or app for the most up-to-date information on the weather in Munich today.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 33, 'prompt_tokens': 17, 'total_tokens': 50, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-BvqQf9B1cLx84SC8KoxZ6CNuvk0xg', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--58df3c21-d13f-49f5-bfe5-05113c3f04f4-0', usage_metadata={'input_tokens': 17, 'output_tokens': 33, 'total_tokens': 50, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

Come vediamo l'LLM non può rispondere a tale domanda dato che l'LLM non ha accesso a informazioni in tempo reale. 

Possiamo dare all'LLM dei tools che ci permettano di collegarsi al mondo esterno e langchain lo rende piuttosto semplice.

---

## 🧩 Step 2 – Definizione degli strumenti

### Metodo 1: Decoratore `@tool`

```python
from langchain.tools import tool

@tool
def check_weather(city: str) -> str:
    """Controlla il meteo per una città.
    
    Args:
        city: Nome della città
    
    Returns:
        Una stringa con il meteo attuale.
    """
    return f"Oggi a {city} il tempo è soleggiato con 22°C."

@tool
def check_outdoor_seating() -> str:
    """Verifica disponibilità dei posti a sedere all'aperto."""
    return "Sono disponibili posti a sedere all'aperto."
```

---

### Metodo 2: Con `Pydantic` (alternativa)

```python
from langchain_core.pydantic_v1 import BaseModel, Field

class WeatherCheck(BaseModel):
    """Check the weather in a specified city."""

    city: str = Field(..., description="Name of the city to check the weather for")


class OutdoorSeatingCheck(BaseModel):
    """Check if outdoor seating is available at a specified restaurant in a given city."""

    city: str = Field(..., description="Name of the city where the restaurant is located")


tools = [WeatherCheck, OutdoorSeatingCheck]
```

---

## 🧩 Step 3 – Binding degli strumenti al LLM

```python
llm_with_tools = llm.bind_tools([check_weather, check_outdoor_seating])
```

In [2]:
# definiamo i tools usando il decoratore @tool
from langchain.tools import tool

@tool
def fake_weather_api(city: str) -> str:
    """
    Check the weather in a specified city.

    Args: 
        city (str): The name of the city where you want to check the weather.

    Returns:
        str: A description of the current weather in the specified city.
    """
    return "Sunny, 22°C"


@tool
def outdoor_seating_availability(city: str) -> str:
    """
    Check if outdoor seating is available at a specified restaurant in a given city.

    Args: 
        city (str): The name of the city where you want to check for outdoor seating availability.

    Returns:
        str: A message stating whether outdoor seating is available or not.
    """

    return "Outdoor seating is available."


tools = [fake_weather_api, outdoor_seating_availability]



In [3]:
llm_with_tools = llm.bind_tools(tools)

In [4]:
result = llm_with_tools.invoke("How will the weather be in munich today?")

result

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_EeVv7ifuF4DNhChGKpDJqTs4', 'function': {'arguments': '{"city":"Munich"}', 'name': 'fake_weather_api'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 172, 'total_tokens': 189, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-BvqpD4dzRKLmCRpklej7PzvRYRtAa', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--1c918183-56e3-47b5-8851-27c02b707477-0', tool_calls=[{'name': 'fake_weather_api', 'args': {'city': 'Munich'}, 'id': 'call_EeVv7ifuF4DNhChGKpDJqTs4', 'type': 'tool_call'}], usage_metadata={'input_tokens': 172, 'output_tokens': 17, 'total_tokens': 189, 'input_token_details': {'audio

**tool_calls** è una lista di dizionari che contengono il nome della funzione che dobbiamo chiamare per rispondere alla domanda e anche i suoi argomenti. 

```python
tool_calls=[{'name': 'fake_weather_api', 'args': {'city': 'Munich'}, 'id': 'call_EeVv7ifuF4DNhChGKpDJqTs4', 'type': 'tool_call'}]
```

A volte, un singolo tool non è sufficiente per rispondere a una domanda. Supponiamo che l'utente ponga la seguente domanda: "How will the weather be in munich today? Do you still have seats outdoor available?"

In questo caso abbiamo bisogno di due tools per dare una risposta concreta

In [5]:
result = llm_with_tools.invoke(
    "How will the weather be in munich today? Do you still have seats outdoor available?"
)

result

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_dhzY82Vnu23OfSVXhtgkXFcZ', 'function': {'arguments': '{"city": "Munich"}', 'name': 'fake_weather_api'}, 'type': 'function'}, {'id': 'call_oRVJ1xjySmZofhTvTQXUxfRE', 'function': {'arguments': '{"city": "Munich"}', 'name': 'outdoor_seating_availability'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 52, 'prompt_tokens': 180, 'total_tokens': 232, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-BwwmrST9NdtGIeq4DfNxMrh5AlUJj', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--656f0fb8-3b95-41da-afa5-5b1e46fdcfd3-0', tool_calls=[{'name': 'fake_weather_api', 'args': {'city': 'Munich'}, 'id': 'call_dhzY82Vn

come vediamo per rispondere a questa domanda il modello ha usato entrambe i tools:

```python
tool_calls=[{'name': 'fake_weather_api', 'args': {'city': 'Munich'}, 'id': 'call_dhzY82Vnu23OfSVXhtgkXFcZ', 'type': 'tool_call'}, {'name': 'outdoor_seating_availability', 'args': {'city': 'Munich'}, 'id': 'call_oRVJ1xjySmZofhTvTQXUxfRE', 'type': 'tool_call'}]
```

In [6]:
result.tool_calls

[{'name': 'fake_weather_api',
  'args': {'city': 'Munich'},
  'id': 'call_dhzY82Vnu23OfSVXhtgkXFcZ',
  'type': 'tool_call'},
 {'name': 'outdoor_seating_availability',
  'args': {'city': 'Munich'},
  'id': 'call_oRVJ1xjySmZofhTvTQXUxfRE',
  'type': 'tool_call'}]

In [7]:
from langchain_core.messages import HumanMessage, ToolMessage

messages = [
    HumanMessage(
        "How will the weather be in munich today? I would like to eat outside if possible"
    )
]

llm_output = llm_with_tools.invoke(messages)
messages.append(llm_output)

In [8]:
messages

[HumanMessage(content='How will the weather be in munich today? I would like to eat outside if possible', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_bStu4zCV03fxds0RFmSzqcNT', 'function': {'arguments': '{"city": "Munich"}', 'name': 'fake_weather_api'}, 'type': 'function'}, {'id': 'call_FA2XKLcX6VcpkeOhYa0WbMyC', 'function': {'arguments': '{"city": "Munich"}', 'name': 'outdoor_seating_availability'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 52, 'prompt_tokens': 180, 'total_tokens': 232, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-Bwx4WtkVTEw6rrAUWETzg1KgXnCu8', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logpr

---

## 🛠️ Step 4 – Tool Calling Manuale

### 1️⃣ Input dell’utente

```python
messages = [
    {"role": "user", "content": "Che tempo farà oggi a Monaco? Avete posti all'aperto?"}
]
```

### 2️⃣ Prima chiamata → Tool selected (ma no output ancora)

```python
ai_response = llm_with_tools.invoke(messages)
# Contiene tool_calls con nome, args e tool_call_id
```

---

## ⚙️ Step 5 – Esecuzione strumenti

### Mapping e chiamata strumenti

```python
tool_map = {
    "check_weather": check_weather,
    "check_outdoor_seating": check_outdoor_seating
}

tool_outputs = []
for call in ai_response.tool_calls:
    tool_fn = tool_map[call["name"]]
    output = tool_fn.invoke(call["args"])
    tool_outputs.append({
        "tool_call_id": call["id"],
        "output": output
    })
```

---

## 🔁 Step 6 – Risposta finale del LLM

### Estensione dei messaggi

```python
messages.append(ai_response)
for result in tool_outputs:
    messages.append({
        "tool_call_id": result["tool_call_id"],
        "role": "tool",
        "content": result["output"]
    })
```

### Chiamata finale

```python
final_response = llm_with_tools.invoke(messages)
```

✅ Output finale:

> “Oggi a Monaco il tempo è soleggiato con una temperatura di 22°C. Sono disponibili posti a sedere all'aperto.”

In [9]:
tool_mapping = {
    "fake_weather_api": fake_weather_api,
    "outdoor_seating_availability": outdoor_seating_availability
}

In [10]:
for tool_call in llm_output.tool_calls:
    tool = tool_mapping[tool_call['name'].lower()]
    tool_output = tool.invoke(tool_call["args"])
    messages.append(ToolMessage(tool_output, tool_call_id=tool_call["id"]))

In [11]:
messages

[HumanMessage(content='How will the weather be in munich today? I would like to eat outside if possible', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_bStu4zCV03fxds0RFmSzqcNT', 'function': {'arguments': '{"city": "Munich"}', 'name': 'fake_weather_api'}, 'type': 'function'}, {'id': 'call_FA2XKLcX6VcpkeOhYa0WbMyC', 'function': {'arguments': '{"city": "Munich"}', 'name': 'outdoor_seating_availability'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 52, 'prompt_tokens': 180, 'total_tokens': 232, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-Bwx4WtkVTEw6rrAUWETzg1KgXnCu8', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logpr

In [12]:
llm_with_tools.invoke(messages)

AIMessage(content='The weather in Munich today is sunny with a temperature of 22°C. You can enjoy eating outside as outdoor seating is available.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 27, 'prompt_tokens': 260, 'total_tokens': 287, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-BwxG2ZyxZixMKoivgNLmvpU9SOSBd', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--c47a7388-18bc-4098-89e6-ecbac81da7bd-0', usage_metadata={'input_tokens': 260, 'output_tokens': 27, 'total_tokens': 287, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

---

## ✅ Vantaggi del Tool Calling

| Vantaggio                      | Descrizione                                              |
| ------------------------------ | -------------------------------------------------------- |
| 🔌 Connessione a mondo esterno | Permette accesso ad API, database, sensori               |
| 🎯 Risposte più affidabili     | Dati aggiornati e pertinenti                             |
| ⚙️ Estensione modulare         | Ogni tool è isolato e riutilizzabile                     |
| 💡 Supporto multi-tool         | Il LLM può richiedere più strumenti in una sola risposta |

---

## 🧠 Considerazioni finali

* `tool_calls` è una **componente nativa** nei messaggi del modello OpenAI.
* LangChain offre un **meccanismo semplice e pulito** per registrarli, invocarli e gestire la risposta.

---

## 📌 Conclusione

Hai imparato a:

* Creare strumenti interattivi per LLM
* Insegnare al modello **quando e come** usare un tool
* Eseguire un ciclo completo di **tool calling**
* Costruire risposte a domande complesse combinando più strumenti

> 🔜 Prossimo step (fuori corso): usare API reali (es. meteo, prenotazioni, ricerca web) per creare **agenti LLM intelligenti**